## LangChain with Simple Chain Pattern
This example is setting up a LangChain example that connects to a custom OpenAI-compatible large language model (LLM) endpoint using credentials from the environment, then prompts it to suggest a provide a 5 lines of overview of "VMware Tanzu Hub"


In [11]:
import sys, os
import requests
import json
import httpx
import warnings
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from cfenv import AppEnv
from tanzu_utils import CFGenAIService
warnings.filterwarnings('ignore')

In [12]:
# Load CF environment
env = AppEnv()

# load your service details replace name with your Gen AI service.  Gen AI service is bound to the app
chat_service = CFGenAIService("tanzu-gpt-oss-120b")

# List available models
models = chat_service.list_models()
for m in models:
    print(f"- {m['name']} (capabilities: {', '.join(m['capabilities'])})")

# construct chat_credentials
chat_credentials = {
    "api_base": chat_service.api_base + "/openai/v1",
    "api_key": chat_service.api_key,
    "model_name": models[0]["name"]
}

- openai/gpt-oss-120b (capabilities: CHAT, TOOLS)


In [13]:
# 2. HTTP client (optional but recommended for custom config)
httpx_client = httpx.Client(verify=False)  # verify=False if your endpoint needs --insecure

# 3. Initialize the LLM
llm = ChatOpenAI(
    temperature=0.9,
    model=chat_credentials["model_name"],   # model name from CF service
    base_url=chat_credentials["api_base"],  # OpenAI-compatible endpoint
    api_key=chat_credentials["api_key"],    # Bearer token
    http_client=httpx_client
)

In [15]:
# Create a prompt template
prompt = ChatPromptTemplate.from_template(
    """Provide a concise 5-line overview of {topic}.
    Focus on its purpose, key capabilities, core components,
    and how it integrates within the VMware Tanzu ecosystem.
    Write in a clear, professional tone suitable for a technical summary."""
)
# Create the chain
chain = LLMChain(llm=llm, prompt=prompt)

# Define the topic
topic = "VMware Tanzu Hub"

# Run the chain
response = chain.run(topic)
print(response)

- **Purpose:** Tanzu Hub serves as a unified control plane that streamlines the provisioning, governance, and lifecycle management of Kubernetes clusters and modern applications across multiple clouds.  
- **Key Capabilities:** Centralized cluster registration, policy‑driven security, automated software supply‑chain (SBOM, signing), and unified observability/monitoring for all Tanzu workloads.  
- **Core Components:** Tanzu Mission Control (cluster management), Tanzu Build Service (image creation), Tanzu Service Mesh (networking), and Tanzu Observability (metrics & logs) are all orchestrated through the Hub’s API‑first console.  
- **Ecosystem Integration:** The Hub connects seamlessly with Tanzu Application Platform, Tanzu Kubernetes Grid, and VMware Cloud Foundation, enabling consistent experience from development to operations.  
- **Value:** By providing a single pane of glass, Tanzu Hub reduces operational complexity, enforces compliance at scale, and accelerates delivery of cloud